In [1]:
import csv

#Load Sample data
with open('./csv/menu_items.csv', mode='r') as file:
    reader = csv.reader(file)

    # Store the anme of menu items in this array.
    documents = []

    # Stores the meta data
    metadatas = []

    # Id for each menu item
    ids = []
    id = 1

    for i, row in enumerate(reader):
        print(i , row)
        # Skip the header row.
        if i == 0:
            continue 
        documents.append(row[1])
        metadatas.append({"item_id": row[0]})
        ids.append(str(id))
        id += 1

0 ['item_id', 'item_name']
1 ['A1', 'Vegan Chicken Salad']
2 ['A2', 'Spring Rolls (4 pieces)']
3 ['A3', 'Pot Stickers (6 pieces)']
4 ['A4', 'Fried Wonton (10 pieces)']
5 ['A5', 'Fried Tofu with Soy Sauce']
6 ['A6', 'Salt and Pepper Tofu']
7 ['A7', 'Vegetable Tempura']
8 ['A11', 'Salt and Pepper Fresh Mushrooms']
9 ['A12', 'Salt and Pepper Vegan Chicken']
10 ['A13', 'Moo Shu Vegetable (6 pieces)']
11 ['A15', 'Salt and Pepper Pumpkin']
12 ['A16', 'Salt and Pepper Green Beans']
13 ['A17', 'Chinese Doughnut']
14 ['A18', 'French Fries']
15 ['B3', 'San Pellegrino']
16 ['B6', 'Thai Ice Almond Milk Tea']
17 ['B7', 'Lemon Ice Tea']
18 ['B8', 'Snapple']
19 ['B14', 'Soda']
20 ['B15', 'Bottled Water']
21 ['B16', 'Strawberry Popping Boba Tea']
22 ['B17', 'Mango Popping Boba Tea']
23 ['B18', 'Lemon Popping Boba Tea']
24 ['B19', 'Passionfruit Popping Boba Tea']
25 ['B21', 'Wintermelon Tea with Popping Boba']
26 ['B22', 'Grapefruit Tea with Popping Boba']
27 ['B24', 'Mango Smoothie']
28 ['B25', 'Straw

In [2]:
import chromadb
from chromadb.utils import embedding_functions

# Instantiate chromadb instance. Data is stored in memory only.
# chroma_client = chromadb.Client()

# Instantiate chromadb instance. Data is stored on disk (a folder named 'my_vectordb' will be created in the same folder as this file).
chroma_client = chromadb.PersistentClient(path="quickstart_files")

In [3]:
# Select the embedding model to use.
# List of model names can be found here https://www.sbert.net/docs/pretrained_models.html
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

# Use this to delete the database
# chroma_client.delete_collection(name="my_collection")

# Create the collection, aka vector database. Or, if database already exist, then use it. Specify the model that we want to use to do the embedding.
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef)

In [4]:
# Add all the data to the vector database. ChromaDB automatically converts and stores the text as vector embeddings. This may take a few minutes.
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [5]:
# Query the vector database

# Query mispelled word: 'vermiceli'. Expect to find the correctly spelled 'vermicelli' item
results = collection.query(
    query_texts=["vermiceli"],
    n_results=5,
    include=['documents', 'distances', 'metadatas']
)
print(results['documents'])

# Query word variation: 'donut'. Expect to find the 'doughnut' item
results = collection.query(
    query_texts=["donut"],
    n_results=5,
    include=['documents', 'distances', 'metadatas']
)
print(results['documents'])

# Query similar meaning: 'shrimp'. Expect to find the 'prawn' items
results = collection.query(
    query_texts=["shrimp"],
    n_results=5,
    include=['documents', 'distances', 'metadatas']
)
print(results['documents'])

[['Melon and Vermicelli', 'Vegetable Tempura', 'Veggie Lee Secret Drink', 'San Pellegrino', 'Assorted Vegetables']]
[['Chinese Doughnut', 'French Fries', 'Strawberry Smoothie', 'Soda', 'Pinenuts with Toons Fried Rice']]
[['Kung Pao Vegan Prawns', 'Salt and Pepper Vegan Prawns', 'Vegan Kung Pao Prawns', 'Vegan Prawns and Mushrooms with Black Pepper Sauce', 'Vegan Prawns with Mixed Nuts']]
